## Deal with datetime and categorical axes

### Datetime axes
Dealing with date and time series is another common task. Bokeh has a sophisticated `DatetimeAxis` that can change the displayed ticks based on the current scale of the plot. There are some inputs for which Bokeh will automatically default to DatetimeAxis, but you can always explicitly ask for one by passing the value "datetime" to the `x_axis_type` or `y_axis_type` parameters to `figure()`. A few things of interest to look out for in this example:

- setting the width and height arguments to figure()
- customizing plots and other objects by assigning values to their attributes
- accessing guides and annotations with convenience Figure attributes: `legend, grid, xgrid, ygrid, axis, xaxis, yaxis`

In [11]:
import bokeh
# bokeh.sampledata.download() download the sample data

In [12]:
from bokeh.sampledata.stocks import AAPL
import numpy as np
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [13]:
# prepare some data
aapl = np.array(AAPL['adj_close'])
aapl_dates = np.array(AAPL['date'], dtype=np.datetime64)

window_size = 30
window = np.ones(window_size)/float(window_size)
aapl_avg = np.convolve(aapl, window, 'same')

# create a new plot with a a datetime axis type
p = figure(width=800, height=350, x_axis_type="datetime")

# add renderers
p.circle(aapl_dates, aapl, size=4, color='darkgrey', alpha=0.2, legend='close')
p.line(aapl_dates, aapl_avg, color='red', legend='avg')

# NEW: customize by setting attributes
p.title.text = "AAPL One-Month Average"
p.legend.location = "top_left"
p.grid.grid_line_alpha=0
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Price'

p.ygrid.band_fill_color="gray"
p.ygrid.band_fill_alpha = 0.1

p.legend.click_policy="hide" # enable the click policy on the legend

# show the results
show(p)

## Categorical axes
To inform Bokeh that the `x-axis` is `categorical`, we pass this list of factors as the `x_range` argument to bokeh plotting figure:

```python
p = figure(x_range=fruits, ... )
```

Note that passing the list of factors is a convenient shorthand notation for creating a FactorRange. The equivalent explicit notation is:
```python
p = figure(x_range=FactorRange(field=fruits), ... )
```
This more explicit for is useful when you want to customize the FactorRange, e.g. by changing the range or category padding.

Next we can call vbar with the list of fruit name factors as the x coordinate, the bar height as the top coordinate, and optionally any width or other properties that we would like to set:
```python
p.vbar(x=fruits, top=[5, 3, 4, 2, 4, 6], width=0.9)
```

Now putting this stuff together we will see the output:

In [14]:
fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']

p = figure(x_range=fruits, plot_height=250, title="Fruit Counts",
           toolbar_location=None, tools="")

p.vbar(x=fruits, top=[5, 3, 4, 2, 4, 6], width=0.6)

p.xgrid.grid_line_color = None
p.y_range.start = 0

show(p)

Often times we may want to have bars that are shaded some color. One way is to supply all the colors up front. This can be done by putting all the data, including the colors for each bar, in a `ColumnDataSource`. Then the name of the column containing the colors is passed to figure as the color (or line_color/fill_color) arguments. This is shown below:

In [15]:
# EXERCISE 1: Create your own simple bar chart

In [17]:
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral6

fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
counts = [5, 3, 4, 2, 4, 6]

source = ColumnDataSource(data=dict(fruits=fruits, counts=counts, color=Spectral6))

p = figure(x_range=fruits, y_range=(0,9), plot_height=250, title="Fruit Counts",
           toolbar_location=None, tools="")

p.vbar(x='fruits', top='counts', width=0.6, color='color', legend="fruits", source=source)

p.xgrid.grid_line_color = None
p.legend.orientation = "horizontal"
p.legend.location = "top_center"

show(p)

In [18]:
# EXERCISE 2: Create your own simple bar chart driven by a ColumnDataSource

### Grouped

When creating bar charts, it is often desirable to visually display the data according to sub-groups. There are two basic methods that can be used, depending on your use case: using nested categorical coordinates, or applying vidual dodges.

### Nested Categories

If the coordinates of a plot range and data have two or three levels, then Bokeh will automatically group the factors on the axis, including a hierarchical tick labeling with separators between the groups. In the case of bar charts, this results in bars grouped together by the top-level factors. This is probably the most common way to achieve grouped bars, especially if you are starting from “tidy” data.

The example below shows this approach by creating a single column of coordinates that are each 2-tuples of the form (fruit, year). Accordingly, the plot groups the axes by fruit type, with a single call to vbar:

In [19]:
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.plotting import figure

fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ['2015', '2016', '2017']

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 3, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

# this creates [ ("Apples", "2015"), ("Apples", "2016"), ("Apples", "2017"), ("Pears", "2015), ... ]
x = [ (fruit, year) for fruit in fruits for year in years ]
counts = sum(zip(data['2015'], data['2016'], data['2017']), ()) # like an hstack in numpy

source = ColumnDataSource(data=dict(x=x, counts=counts))

p = figure(x_range=FactorRange(*x), plot_height=250, title="Fruit Counts by Year",
           toolbar_location=None, tools="")

p.vbar(x='x', top='counts', width=0.9, source=source)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

show(p)

In [21]:
# EXERCISE 3: Make the chart above have a different color for each year by adding colors to the ColumnDataSource

We can also apply a color mapping, similar to the earlier example. To obtain same grouped bar plot of fruits data as above, except with the bars shaded by the year, changethe vbar function call to use `factor_cmap` for the fill_color:

In [20]:
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral10

# use the palette to colormap based on the the x[1:2] values
p.vbar(x='x', top='counts', width=0.9, source=source, line_color="white",
       fill_color=factor_cmap('x', palette=Spectral10, factors=years, start=1, end=2))
show(p)

Another method for achieving grouped bars is to explicitly specify a visual displacement for the bars. Such a visual offset is also referred to as a dodge.

In this scenario, our data is not “tidy”. Instead a single table with rows indexed by factors (fruit, year), we have separate series for each year. We can plot all the year series using separate calls to vbar but since every bar in each group has the same fruit factor, the bars would overlap visually. We can prevent this overlap and distinguish the bars visually by using the dodge() function to provide an offset for each different call to vbar:

In [22]:
from bokeh.core.properties import value
from bokeh.models import ColumnDataSource
from bokeh.transform import dodge

fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ['2015', '2016', '2017']

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 3, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

source = ColumnDataSource(data=data)

p = figure(x_range=fruits, y_range=(0, 10), plot_height=250, title="Fruit Counts by Year",
           toolbar_location=None, tools="")

p.vbar(x=dodge('fruits', -0.25, range=p.x_range), top='2015', width=0.2, source=source,
       color="#c9d9d3", legend=value("2015"))

p.vbar(x=dodge('fruits',  0.0,  range=p.x_range), top='2016', width=0.2, source=source,
       color="#718dbf", legend=value("2016"))

p.vbar(x=dodge('fruits',  0.25, range=p.x_range), top='2017', width=0.2, source=source,
       color="#e84d60", legend=value("2017"))

p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

show(p)

Another common operation or bar charts is to stack bars on top of one another. Bokeh makes this easy to do with the specialized hbar_stack() and vbar_stack() functions. The example below shows the fruits data from above, but with the bars for each fruit type stacked instead of grouped:

In [23]:
fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ["2015", "2016", "2017"]
colors = ["#34A212", "#D3F543", "#98F5FF"]

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 4, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

source = ColumnDataSource(data=data)

p = figure(x_range=fruits, plot_height=250, title="Fruit Counts by Year",
           toolbar_location=None, tools="")

p.vbar_stack(years, x='fruits', width=0.9, color=colors, source=source,
             legend=[value(x) for x in years])

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

show(p)

Sometimes we may want to stack bars that have both positive and negative extents. The example below shows how it is possible to create such a stacked bar chart that is split by positive and negative values:

In [24]:
from bokeh.palettes import GnBu3, OrRd3

fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ["2015", "2016", "2017"]

exports = {'fruits' : fruits,
           '2015'   : [2, 1, 4, 3, 2, 4],
           '2016'   : [5, 3, 4, 2, 4, 6],
           '2017'   : [3, 2, 4, 4, 5, 3]}
imports = {'fruits' : fruits,
           '2015'   : [-1, 0, -1, -3, -2, -1],
           '2016'   : [-2, -1, -3, -1, -2, -2],
           '2017'   : [-1, -2, -1, 0, -2, -2]}

p = figure(y_range=fruits, plot_height=250, x_range=(-16, 16), title="Fruit import/export, by year",
           toolbar_location=None)

p.hbar_stack(years, y='fruits', height=0.9, color=GnBu3, source=ColumnDataSource(exports),
             legend=["%s exports" % x for x in years])

p.hbar_stack(years, y='fruits', height=0.9, color=OrRd3, source=ColumnDataSource(imports),
             legend=["%s imports" % x for x in years])

p.y_range.range_padding = 0.1
p.ygrid.grid_line_color = None
p.legend.location = "top_left"
p.axis.minor_tick_line_color = None
p.outline_line_color = None

show(p)

## Mixing Categorical Levels

If you have created a range with nested categories as above, it is possible to plot glyphs using only the "outer" categories, if desired. The plot below shows monthly values grouped by quarter as bars. The data for these are in the famliar format:

    factors = [("Q1", "jan"), ("Q1", "feb"), ("Q1", "mar"), ....]

The plot also overlays a line representing average quarterly values, and this is accomplished by using only the "quarter" part of each nexted category:

    p.line(x=["Q1", "Q2", "Q3", "Q4"], y=....)

In [25]:
factors = [("Q1", "jan"), ("Q1", "feb"), ("Q1", "mar"),
           ("Q2", "apr"), ("Q2", "may"), ("Q2", "jun"),
           ("Q3", "jul"), ("Q3", "aug"), ("Q3", "sep"),
           ("Q4", "oct"), ("Q4", "nov"), ("Q4", "dec")]

p = figure(x_range=FactorRange(*factors), plot_height=250)

x = [ 10, 12, 16, 9, 10, 8, 12, 13, 14, 14, 12, 16 ]
p.vbar(x=factors, top=x, width=0.9, alpha=0.5)

qs, aves = ["Q1", "Q2", "Q3", "Q4"], [12, 9, 13, 14]
p.line(x=qs, y=aves, color="red", line_width=3)
p.circle(x=qs, y=aves, line_color="red", fill_color="white", size=10)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None

show(p)

## Using Pandas `GroupBy`

We may want to make charts based on the results of "group by" operations. Bokeh can utilize Pandas `GroupBy` objects directly to make this simpler. Let's take a look at how Bokeh deals with `GroupBy` objects by examining the "cars" data set.

In [27]:
from bokeh.sampledata.autompg import autompg_clean as df

df.cyl = df.cyl.astype(str)
df.head()

,mpg,cyl,displ,hp,weight,accel,yr,origin,name,mfr
0,18.0,8,307.0,130,3504,12.0,70,North America,chevrolet chevelle malibu,chevrolet
1,15.0,8,350.0,165,3693,11.5,70,North America,buick skylark 320,buick
2,18.0,8,318.0,150,3436,11.0,70,North America,plymouth satellite,plymouth
3,16.0,8,304.0,150,3433,12.0,70,North America,amc rebel sst,amc
4,17.0,8,302.0,140,3449,10.5,70,North America,ford torino,ford


Suppose we would like to display some values grouped according to `"cyl"`. If we create `df.groupby(('cyl'))` then call `group.describe()` we can see that Pandas automatically computes various statistics for each group. 

In [28]:
group = df.groupby(('cyl'))

group.describe()

accel                                                      displ  \
     count       mean       std   min    25%   50%   75%   max  count   
cyl                                                                     
3      4.0  13.250000  0.500000  12.5  13.25  13.5  13.5  13.5    4.0   
4    199.0  16.581910  2.383185  11.6  14.80  16.2  18.0  24.8  199.0   
5      3.0  18.633333  2.369247  15.9  17.90  19.9  20.0  20.1    3.0   
6     83.0  16.254217  2.031778  11.3  15.05  16.0  17.6  21.0   83.0   
8    103.0  12.955340  2.224759   8.0  11.50  13.0  14.0  22.2  103.0   

                 ...   weight             yr                             \
           mean  ...      75%     max  count       mean       std   min   
cyl              ...                                                      
3     72.500000  ...   2495.0  2720.0    4.0  75.500000  3.696846  72.0   
4    109.670854  ...   2562.5  3270.0  199.0  77.030151  3.737484  70.0   
5    145.000000  ...   3240.0  3530.0    3.0  79.000000  1.000000  78.0   
6    218.361446  ...   3431.0  3907.0   83.0  75.951807  3.264381  70.0   
8    345.009709  ...   4403.5  5140.0  103.0  73.902913  3.021214  70.0   

                               
       25%   50%    75%   max  
cyl                            
3    72.75  75.0  77.75  80.0  
4    74.00  77.0  80.00  82.0  
5    78.50  79.0  79.50  80.0  
6    74.00  76.0  78.00  82.0  
8    72.00  73.0  76.00  81.0  

[5 rows x 48 columns]

Bokeh allows us to create a `ColumnDataSource` directly from Pandas `GroupBy` objects, and when this happens, the data source is automatically filled with the summary values from `group.desribe()`. Observe the column names below, which correspond to the output above.

In [30]:
source = ColumnDataSource(group)

",".join(source.column_names)

'accel_count,accel_mean,accel_std,accel_min,accel_25%,accel_50%,accel_75%,accel_max,displ_count,displ_mean,displ_std,displ_min,displ_25%,displ_50%,displ_75%,displ_max,hp_count,hp_mean,hp_std,hp_min,hp_25%,hp_50%,hp_75%,hp_max,mpg_count,mpg_mean,mpg_std,mpg_min,mpg_25%,mpg_50%,mpg_75%,mpg_max,weight_count,weight_mean,weight_std,weight_min,weight_25%,weight_50%,weight_75%,weight_max,yr_count,yr_mean,yr_std,yr_min,yr_25%,yr_50%,yr_75%,yr_max,cyl'

Knowing these column names, we can immediately create bar charts based on Pandas `GroupBy` objects. The example below plots the aveage MPG per cylinder, i.e. columns `"mpg_mean"` vs `"cyl"`

In [31]:
from bokeh.palettes import Spectral5

cyl_cmap = factor_cmap('cyl', palette=Spectral5, factors=sorted(df.cyl.unique()))

p = figure(plot_height=350, x_range=group)
p.vbar(x='cyl', top='mpg_mean', width=1, line_color="white", 
       fill_color=cyl_cmap, source=source)

p.xgrid.grid_line_color = None
p.xaxis.axis_label = "number of cylinders"
p.yaxis.axis_label = "Mean MPG"
p.y_range.start = 0

show(p)

In [32]:
# EXERCISE 4: Use the same dataset to make a similar plot of mean horsepower (hp) by origin